# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [84]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact

In [85]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [86]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [87]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path).set_index("neighbourhood")

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [97]:

to_most_expensive=to_data.groupby('neighbourhood').mean().nlargest(10,'average_house_value')

to_data_per_year=to_data.drop(to_data.columns[-3:],axis=1).groupby('year').sum()

to_data_ave_cost=to_data[["shelter_costs_owned","shelter_costs_rented"]].groupby('year').mean()

average_house_val_per_year=to_data["average_house_value"].groupby('year').mean()

average_house_val_by_neighbourhood_per_year=to_data[["neighbourhood","average_house_value"]].reset_index()

to_data_dwelling_types=to_data.drop(to_data.columns[-3:],axis=1).reset_index()

to_avprice=to_data.groupby('neighbourhood').mean()

to_data_with_loc=pd.concat([df_neighbourhood_locations,to_avprice],join='outer',axis=1).reset_index()

to_data_index=to_data.reset_index()

to_most_expensive_sun=pd.concat([to_data_index.loc[to_data_index['year']==2001].nlargest(10,'average_house_value'),
                             to_data_index.loc[to_data_index['year']==2006].nlargest(10,'average_house_value'),
                             to_data_index.loc[to_data_index['year']==2011].nlargest(10,'average_house_value'),
                             to_data_index.loc[to_data_index['year']==2016].nlargest(10,'average_house_value'),
                            ],
                            join='outer').reset_index(drop=True)


### Panel Visualization Functions

In [113]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
   
    map1 = px.scatter_mapbox(
        to_data_with_loc,
        lat="lat",
        lon="lon",
        size="average_house_value",
        color="average_house_value",
        zoom=10,
        width=1000)
    return map1
#@interact(year=[2001,2006,2011,2016])
def create_bar_chart(Year):
    """
    Create a barplot based in the data argument.
    """
    year=int(Year)
    color_schema={2001:'red',
                 2006:'blue',
                 2011:'yellow',
                 2016:'purple'}
    aux_df=to_data_per_year.loc[year].iloc[0:8]
    return aux_df.hvplot(kind="bar",
                         title=f'Bar chart for {year}',
                         height=500,
                         color=color_schema[year],
                         ylabel="Dwelling type units",
                         xlabel=str(year),
                         rot=90).opts(yformatter="%.0f")

                    

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    
    return data.hvplot(height=500,
                       width=600,
                     title=title,
                     color=color,
                     xlabel=xlabel, 
                     ylabel=ylabel)

def average_house_value():
    """Average house values per year."""
    
    return average_house_val_per_year.hvplot().opts(yformatter="%.0f")

    

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    return average_house_val_by_neighbourhood_per_year.hvplot(x='year',
                                                              y='average_house_value',
                                                              groupby='neighbourhood',
                                                              xlabel='Year',
                                                              ylabel='Avg. House Value').opts(yformatter="%.0f")

def number_dwelling_types():
    """Number of dwelling types per year"""
    
    return to_data_dwelling_types.hvplot.bar(x='year',
                                             groupby='neighbourhood',
                                             rot=90,
                                             height=500,
                                             xlabel='Year',
                                             ylabel='Dwelling Type Units').opts(yformatter="%.0f")

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""

    return px.bar(to_data_index,
                  width=1000,
                  x='neighbourhood',
                  y='average_house_value',
                  facet_row='year',
                  color='average_house_value')

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    return to_most_expensive.hvplot.bar(y='average_house_value',
                                        rot=90,
                                        height=500,
                                        xlabel='Neighbourhood',
                                        ylabel='Avg.House Value',
                                        title='Top 10 Expensive Neighbourhoods in Toronto',
                                        dynamic=True).opts(framewise=True)

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    return px.sunburst(to_most_expensive_sun,
                             path=['year','neighbourhood'],
                             values='average_house_value',
                             color='average_house_value',
                             color_continuous_scale='RdBu'
                             )

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [115]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
dashboard = pn.GridSpec(sizing_mode='stretch_both') 
dashboard = pn.Column("# Toronto Housing Market")


# Define a welcome text
dashboard.append("## Statiscs over 2001,2006,2011 and 2016")


to_volume=pn.Row(interact(create_bar_chart,Year=['2001','2006','2011','2016']))
to_cost=pn.Row(create_line_chart(to_data_ave_cost["shelter_costs_owned"],"Average Monthly Shelter Cost for Owned Dwellings in Toronto","Year","Avg Monthly Shelter Costs",'b'),
               create_line_chart(to_data_ave_cost["shelter_costs_rented"],"Average Monthly Shelter Cost for Rented Dwellings in Toronto","Year","Avg Monthly Shelter Costs",'r')
               )
to_av_val=pn.Row(average_house_value())

to_stats_tabs=pn.Tabs(("Dwelling Volume",to_volume),
                     ("Costs",to_cost),
                     ("Average House Value",to_av_val))


nei_stats_tabs=pn.Tabs(("Average Value",pn.Row(average_value_by_neighbourhood())),
                       ("Average Dwelling Volume",pn.Row(average_house_value_snapshot())),
                       ("Value Map",pn.Row(neighbourhood_map()))
                      )

most_exp_tabs=pn.Tabs(("Top Most Expensive",pn.Row(top_most_expensive_neighbourhoods())),
                      ("Sunburst Cost",pn.Row(sunburts_cost_analysis()))
                     )

tabs=pn.Tabs(("Toronto Statistics",to_stats_tabs),
             ("Neighbourhood Statistics",nei_stats_tabs),
             ("Most Expensive Top 10",most_exp_tabs))

dashboard.append(tabs)
dashboard


Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Tabs
        [0] Tabs
            [0] Row
                [0] Column
                    [0] Column
                        [0] Select(name='Year', options=['2001', '2006', ...], value='2001')
                    [1] Row
                        [0] HoloViews(Bars, name='interactive41003')
            [1] Row
                [0] HoloViews(Curve)
                [1] HoloViews(Curve)
            [2] Row
                [0] HoloViews(Curve)
        [1] Tabs
            [0] Row
                [0] Row
                    [0] HoloViews(DynamicMap)
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                        [1] VSpacer()
            [1] Row
                [0] Plotly(Figure)
            [2] Row
                [0] Plotly(Figure)
        [2] Tabs
            [0] Row
                [0] HoloViews(Bars)
            [1] Row
                [0] Plotly(Figure)

## Serve the Panel Dashboard

In [116]:
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Tabs
        [0] Tabs
            [0] Row
                [0] Column
                    [0] Column
                        [0] Select(name='Year', options=['2001', '2006', ...], value='2001')
                    [1] Row
                        [0] HoloViews(Bars, name='interactive41003')
            [1] Row
                [0] HoloViews(Curve)
                [1] HoloViews(Curve)
            [2] Row
                [0] HoloViews(Curve)
        [1] Tabs
            [0] Row
                [0] Row
                    [0] HoloViews(DynamicMap)
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                        [1] VSpacer()
            [1] Row
                [0] Plotly(Figure, viewport={'xaxis.range': [-0.5, ...})
            [2] Row
                [0] Plotly(Figure)
        [2] Tabs
            [0] Row
                [0] HoloViews(Bars)
            [1] Row
                [0] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [19]:
# neighbourhood_map().show()

In [54]:


# # Bar chart for 2001

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [12]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [13]:
# average_house_value()

In [14]:
# average_value_by_neighbourhood()

In [15]:
# number_dwelling_types()

In [16]:
# average_house_value_snapshot()

In [17]:
# top_most_expensive_neighbourhoods()

In [18]:
# sunburts_cost_analysis()